In [1]:
# ************************************************************
# Main file in cytokine-identification process. 
#
# Master Thesis, KTH , Gabriel Andersson 
# ************************************************************
import sys
import os,sys,inspect

sys.path.insert(1,'../')

import numpy as np
import matplotlib.pyplot as plt
from plot_functions_wf import *
from main_functions import *
from sklearn.cluster import KMeans, DBSCAN
from event_rate_first import *


In [2]:
# ************************************************************
# ******************** Parameters ****************************
# ************************************************************

# VAE training params:
continue_train = False
nr_epochs = 100 # if all train data is used -- almost no loss-decrease after 100 batches..
batch_size = 128


view_vae_result = False # True => reqires user to give input if to continue the script to pdf-GD or not.. 
view_GD_result = False # This reqires user to give input if to continue the script to clustering or not.

run_DBscan = False
run_KMeans = False


verbose = 2

# pdf GD params: 
m=0 # Number of steps 
gamma=0.01 # learning_rate
eta=0.01 # Noise variable -- adds white noise with variance eta to datapoints during GD.


# DBSCAN params
db_eps = 0.2 # max_distance to be considered as neighbours 
db_min_sample = 200 # Minimum members in neighbourhood to not be regarded as Noise.

# Shape of waveforms: (136259, 141)
#training_idx = np.arange(10000) # initial testing
training_idx = np.arange(0,131000,10)

# ************************************************************
# ******************** Paths ****************************
# ************************************************************

path_to_wf = '../../matlab_files/gg_waveforms-R10_IL1B_TNF_03.mat' 
path_to_ts = '../../matlab_files/gg_timestamps.mat'

save_figure = '../figures/deleteme_23nov'
# tf weight-file:
path_to_weights = '../models/deleteme_23nov'
# Numpy file:
path_to_hpdp = "../../numpy_files/numpy_hpdp/deleteme_23nov"

# ************************************************************
# ******************** Load Files ****************************
# ************************************************************
load_data = True
if load_data:
    waveforms, mean, std = load_waveforms(path_to_wf,'waveforms',standardize=True, verbose=1)
    timestamps = load_timestamps(path_to_ts,'gg_timestamps',verbose=1)
    
    # Extract Training data:
    wf_train = waveforms[training_idx]
    ts_train = timestamps[training_idx]
    #wf_train = waveforms
    #ts_train = timestamps
    print(f'Shape of training data: {wf_train.shape}')


Loading matlab waveforms files...

waveforms loaded succesfully...

Shape of waveforms: (136259, 141).

Standardize waveforms...


Loading matlab timestamps file...

timestamps loaded succesfully...

Shape of timestamps: (136259, 1).
Shape of training data: (13100, 141)


In [3]:
continue_train = True

waveform_shape = waveforms.shape[-1]
encoder,decoder,gmm_vae = get_gmm_vae(waveform_shape,2)

history = gmm_vae.fit(wf_train, epochs=nr_epochs, batch_size=128,verbose=1)
gmm_vae.save_weights(saved_weights)
plt.plot(history.history['loss'])
plt.show() 
print()
print(f'Loading {path_to_weights}...')
print()
vae.load_weights(path_to_weights)
if continue_train == True:
    print()
    print(f'Continue training for {nr_epochs} epochs...')
    print()
    history = gmm_vae.fit(wf_train, epochs=nr_epochs, batch_size=128,verbose=1)
    gmm_vae.save_weights(path_to_weights)
    plt.plot(history.history['loss'])
    plt.show() 
#print()
#print(f'Visualising decoded latent space...')
#print()
#plot_decoded_latent(decoder,saveas=save_figure+'_decoded',verbose=1)
#plot_encoded(encoder, x_train, saveas=None, verbose=1)
z_mean,_,zz = encoder.predict(x_train)

'''
gmm_components = 5
Gmm = Latent_GMM(gmm_components,covariance_type='full')
Gmm.fit(zz)
g_means = Gmm.get_gaussian_means()
g_var = Gmm.get_gaussian_var()
probs = Gmm.soft_assignment(zz)
labels = Gmm.classify(zz)

def plot_gmm(data,labels):
    print()
    print(f'Visualising gaussian mixture model...')
    print()
    for ii in range(gmm_components):
        plt.scatter(data[labels==ii][::2,0],data[labels==ii][::2,1])

    plt.show()

plot_gmm(zz,labels)

print('Done.')
'''

Epoch 1/100
103/103 [==============================] - 0s 3ms/step - loss: 96.1744 - reconstruction_loss: 94.1870 - kl_loss: 1.9874
Epoch 2/100
103/103 [==============================] - 0s 3ms/step - loss: 81.1388 - reconstruction_loss: 79.2081 - kl_loss: 1.9307
Epoch 3/100
103/103 [==============================] - 0s 3ms/step - loss: 79.1001 - reconstruction_loss: 77.1001 - kl_loss: 1.9999
Epoch 4/100
103/103 [==============================] - 0s 3ms/step - loss: 77.9043 - reconstruction_loss: 75.8338 - kl_loss: 2.0705
Epoch 5/100
103/103 [==============================] - 0s 3ms/step - loss: 77.0231 - reconstruction_loss: 74.8990 - kl_loss: 2.1240
Epoch 6/100
103/103 [==============================] - 0s 3ms/step - loss: 76.6891 - reconstruction_loss: 74.5407 - kl_loss: 2.1483
Epoch 7/100
103/103 [==============================] - 0s 4ms/step - loss: 76.2974 - reconstruction_loss: 74.1176 - kl_loss: 2.1798
Epoch 8/100
103/103 [==============================] - 0s 3ms/step - loss: 7

NameError: name 'saved_weights' is not defined

## OBS the below is not compatible with GMM-syntax yet..:

In [ ]:
# ************************************************************
# ******************** Train/Load model **********************
# ************************************************************
print()
print('*********************** Tensorflow Blaj *************************************')
print()

encoder,decoder,vae = (wf_train, nr_epochs=nr_epochs, batch_size=batch_size, path_to_weights=path_to_weights, 
                                        continue_train=continue_train, verbose=1)
print()
print('******************************************************************************')
print()

#view_vae_result = False # This reqires user to give input if to continue the script to GD or not.
if view_vae_result:
    plot_decoded_latent(decoder,saveas=save_figure+'_decoded',verbose=1)
    continue_to_run_GD = input('Continue to gradient decent of pdf? (yes/no) :')

    all_fine = False
    while all_fine==False:
        if continue_to_run_GD=='no':
            exit()
        elif continue_to_run_GD=='yes':
            print('Continues to "run_GD"')
            all_fine = True
        else:
            continue_to_run_GD = input('Invalid input, continue to gradient decent of pdf? (yes/no) :')


# ************************************************************
# ** Perform GD on pdf to find high prob. data-points (hpdp) *
# ************************************************************  

run_GD = True
#view_GD_result = True # This reqires user to give input if to continue the script to clustering or not.
if run_GD:
    print()
    print('Running pdf_GD to get hpdp...')
    print()
    # To easy computational load -- only every 20th data-point is used..
    hpdp = pdf_GD(vae, wf_train[0::20], m=m, gamma=gamma, eta=eta, path_to_hpdp=path_to_hpdp,verbose=verbose)
    #hpdp = pdf_GD(vae, wf_train, m=m, gamma=gamma, eta=eta, path_to_hpdp=path_to_hpdp,verbose=verbose)

    if view_GD_result:
        print(f'Visualising decoded latent space of hpdp...')
        print()
        plot_encoded(encoder, hpdp, saveas=save_figure+'_hpdp_encoded', verbose=1)        
        continue_to_Clustering = input('Continue to Clustering? (yes/no) :')

        all_fine = False
        while all_fine==False:
            if continue_to_Clustering=='no':
                exit()
            elif continue_to_Clustering=='yes':
                print('Continues to "run_GD"')
                all_fine = True
            else:
                continue_to_Clustering = input('Invalid input, continue to Clustering? (yes/no) :')

else:
    print()
    print('Skipps over pdf_GD...')
    print()



# ************************************************************
# ******************** Cluster wf using hpdp *****************
# ******************** This to access labels *****************
# ************************************************************


if run_KMeans:
    print()
    print('Running KMeans on hpdp...')
    print()
    kmeans = KMeans(n_clusters=10, random_state=0).fit(hpdp)
    labels = kmeans.labels_
else:
    print()
    print('Skipps over KMeans...')
    print()

if run_DBscan:
    print()
    print('Running DBSCAN on hpdp...')
    print()
    dbscan = DBSCAN(eps=db_eps, min_samples=db_min_sample, metric='euclidean', metric_params=None, algorithm='auto', leaf_size=30, p=None, n_jobs=None)
    hpdp_latent_mean,_,_ = encoder.predict(hpdp)
    dbscan.fit(hpdp_latent_mean)
    labels = dbscan.labels_ #  Noisy samples are given the label -1
else:
    print()
    print('Skipps over DBSCAN...')
    print()


# ************************************************************
# ******************** Event Rate *****************
# ************************************************************
run_event_rate = True
if run_event_rate:
    print('Calculating Event rates...')
    event_rates, real_clusters = get_event_rates(ts_train[0::20],labels,bin_width=1)
    delta_ev, ev_stats = delta_ev_measure(event_rates,timestamps,labels)

    #event_rates, real_clusters = get_event_rates(ts_train,labels,bin_width=1)
    print(f'Real cluster (with mean event_rate over 0.5 is CAPs {real_clusters})')
    plot_event_rates(event_rates,ts_train[0::20],saveas=save_figure+'_event_rate', conv_width=30)
    #plot_event_rates(event_rates,ts_train,saveas=save_figure+'_event_rate', conv_width=30)

# ************************************************************
# ******************** General PLOTTING ******************************
# ************************************************************
# Plot hpdp:
if verbose>1:
    print()
    print(f'Plotting waveforms of each cluster if labels are specified...')
    print()
    plot_waveforms(hpdp,labels=None)
    print()
    print(f'Visualising decoded latent space of hpdp...')
    print()
    plot_encoded(encoder, hpdp, saveas=save_figure+'_hpdp_encoded', verbose=verbose)
    print()
    print(f'Visualising decoded latent space...')
    print()
    plot_decoded_latent(decoder,saveas=save_figure+'_decoded',verbose=1)
    plot_encoded(encoder, wf_train, saveas=save_figure+'_encoded', verbose=1)